# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3
# download necessary NLTK data
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download(['punkt', 'wordnet'])

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName','sqlite:///InsertDatabaseName.db')
X = df['message']
Y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # tokenize text
    words = word_tokenize(text)
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    # iterate through each token
    clean_tokens = []
    for tok in words:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens
    #pass

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [5]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [8]:
# iterate over columns and print classification report for each
for i, col in enumerate(y_test.columns):
    print(f"Category: {col}")
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

Category: related
             precision    recall  f1-score   support

          0       0.60      0.36      0.45      1562
          1       0.81      0.93      0.87      4943
          2       0.43      0.06      0.11        49

avg / total       0.76      0.78      0.76      6554

Category: request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5400
          1       0.83      0.37      0.51      1154

avg / total       0.87      0.88      0.86      6554

Category: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6529
          1       0.00      0.00      0.00        25

avg / total       0.99      1.00      0.99      6554

Category: aid_related
             precision    recall  f1-score   support

          0       0.72      0.88      0.79      3851
          1       0.75      0.51      0.61      2703

avg / total       0.73      0.73      0.72      6554

Category: med

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'clf__estimator__n_estimators': [10, 20],
    'clf__estimator__min_samples_split': [2, 4],
}

cv = GridSearchCV(pipeline, param_grid=parameters,cv=3,n_jobs=-1)


In [ ]:
model=cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = cv.predict(X_test)

In [ ]:
# iterate over columns and print classification report for each
for i, col in enumerate(y_test.columns):
    print(f"Category: {col}")
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
def build_model():
    """
    Build and return a GridSearchCV object using a pipeline that preprocesses text data and fits a random forest 
    classifier to classify multiple target labels. Includes an additional feature transformer for text length.

    Returns:
    cv (GridSearchCV): A GridSearchCV object with the specified pipeline and parameters.
    """
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
            ])),
            ('text_length', FunctionTransformer(get_text_length, validate=False)),
        ])),
        ('clf', MultiOutputClassifier(LogisticRegression()))
    ])
    parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'clf__estimator__C': [0.5, 1.0,1.5],
        'clf__estimator__solver': ['lbfgs', 'newton-cg','sag','saga'],
    }
    cv = GridSearchCV(pipeline, param_grid=parameters, cv=5, n_jobs=-1)
    return cv

def display_results(cv, y_test, y_pred):
    """
    Display the accuracy and best parameters of a trained model using the provided test targets and predicted targets.
    
    Args:
    cv (GridSearchCV): A trained GridSearchCV model.
    y_test (pandas DataFrame): The test targets.
    y_pred (numpy ndarray): The predicted targets.
    """
    labels = np.unique(y_pred)
    accuracy = (y_pred == y_test).mean()
    print("Labels:", labels)
    print("Accuracy:", accuracy)
    print("\nBest Parameters:", cv.best_params_)


In [ ]:
model = build_model()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
display_results(model, y_test, y_pred)


### 9. Export your model as a pickle file

In [ ]:
# Save the trained model as a pickle file
with open('trained_model.pkl', 'wb') as file:
    pickle.dump(model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
# import packages
import sys
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3
# download necessary NLTK data
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download(['punkt', 'wordnet'])

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

def load_data(data_file):
    # read in file
    messages = pd.read_csv("messages.csv")
    categories = pd.read_csv("categories.csv")
    df = messages.merge(categories, on="id")
    
    # clean data
    categories = df.categories.str.split(";", expand=True)
    row = categories.iloc[0,:]
    category_colnames = row.str.split("-", n=1,expand=True)[0]
    categories.columns = category_colnames
    for column in categories:
    categories[column] = categories[column].apply(lambda x: x[-1])
    categories[column] = categories[column].astype(int)
    df=df.drop(['categories'], axis=1)
    df = pd.merge(df,categories,left_index=True,right_index=True)
    df=df.drop_duplicates()
    words = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in words:
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    # load to database
    engine = create_engine('sqlite:///CleanData.db')
    df.to_sql('CleanData', engine, index=False)

    # define features and label arrays
    engine = create_engine('sqlite:///InsertDatabaseName.db')
    df = pd.read_sql_table('InsertTableName','sqlite:///InsertDatabaseName.db')
    X = df['message']
    Y = df.iloc[:, 4:]

    return X, y


def build_model():
    def build_model():
    """
    Build and return a GridSearchCV object using a pipeline that preprocesses text data and fits a random forest 
    classifier to classify multiple target labels. Includes an additional feature transformer for text length.

    Returns:
    cv (GridSearchCV): A GridSearchCV object with the specified pipeline and parameters.
    """
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
            ])),
            ('text_length', FunctionTransformer(get_text_length, validate=False)),
        ])),
        ('clf', MultiOutputClassifier(LogisticRegression()))
    ])
    parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'clf__estimator__C': [0.5, 1.0,1.5],
        'clf__estimator__solver': ['lbfgs', 'newton-cg','sag','saga'],
    }
    model_pipeline = GridSearchCV(pipeline, param_grid=parameters, cv=5, n_jobs=-1)


    return model_pipeline


def train(X, y, model):
    # train test split
    X_train, X_test, y_train, y_test = train_test_split(X, Y)

    # fit model
    model=model_pipeline.fit(X_train, y_train)
    # output model test results
    y_pred = model.predict(X_test)

    return model


def export_model(model):
    # Export model as a pickle file
    with open('trained_model.pkl', 'wb') as file:
    pickle.dump(model, file)

def run_pipeline(data_file):
    X, y = load_data(data_file)  # run ETL pipeline
    model = build_model()  # build model pipeline
    model = train(X, y, model)  # train model pipeline
    export_model(model)  # save model


if __name__ == '__main__':
    data_file = sys.argv[1]  # get filename of dataset
    run_pipeline(data_file)  # run data pipeline